In [1]:
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse
from qdrant_client.models import  SparseVector
from qdrant_client import QdrantClient, models
from langchain import PromptTemplate    
from langchain.schema import BaseMemory
from langchain.memory import ChatMessageHistory

import numpy as np
import pandas as pd
import time
import json
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import Faithfulness

import pandas as pd
import json
import ast 
import csv
import os
import time
import ast


In [2]:
# Load environment variables from .env file
load_dotenv()

# Access OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Debug: Check if the API key is being correctly loaded
if openai_api_key is None or openai_api_key == "":
    print("OpenAI API Key not found! Please make sure it is set correctly in the .env file.")
else:
    print(f"OpenAI API Key Loaded: {openai_api_key[:4]}...")  # Only print part of the key for security reasons


llmName = ChatOpenAI(model="gpt-4o")
Chat_llm = ChatOpenAI(model="gpt-4o", temperature=1)



dense_embedding_model = OpenAIEmbeddings(
                api_key=openai_api_key,
                model="text-embedding-3-large"
            )
sparse_embedding_model = FastEmbedSparse(model_name="Qdrant/bm25")
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

OpenAI API Key Loaded: sk-p...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# Link to the Qdrant database & defines name for the collections
Clientpath = "VectorDB/CombinedData"
client = QdrantClient(path=Clientpath)
merged_collection_name = "merged vectors collection"

In [ ]:
# Define the function for sparse and dense retrieval
def merged_dense_for_RRF(user_question, limitN):
    dense_query_vector =  dense_embedding_model.embed_query(user_question)
    merged_dense = client.query_points(
        merged_collection_name,
        query=dense_query_vector,
        using="text-embedding-3-large",
        limit=limitN,
        with_payload=True, 
    )

    return merged_dense

def merged_sparse_for_RRF(user_question, limitN):
    sparse_query_vector = sparse_embedding_model.embed_query(user_question)
    merged_sparse = client.query_points(
        collection_name=merged_collection_name,
        query=SparseVector(indices=sparse_query_vector.indices, values=sparse_query_vector.values),  
        using="bm25",
        with_payload=True,  
        limit=limitN  
    )
    
    return merged_sparse



In [ ]:
# RRF calculation for ids
def reciprocal_rank_fusion(results1, results2, k=60):
    """Combine results using Reciprocal Rank Fusion (RRF)."""
    rrf_scores = {}

    # Process results
    for rank, scored_point in enumerate(results1.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    for rank, scored_point in enumerate(results2.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    # Sort by the combined RRF score
    sorted_results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_results



In [ ]:
# From id back to list
def Fetch_chunks_back(fused_results):    
    fused_ids = [doc_id for doc_id, _ in fused_results]


    detailed_results = client.retrieve(
        collection_name=merged_collection_name,
        ids=fused_ids,
        with_payload=True,  # Include payload in the results
        with_vectors=False,  # Optional: Include vectors if needed
    )

    detailed_results_with_scores = []
    for result in detailed_results:
        # print(result)
        doc_id = result.id
        score = next((s for d, s in fused_results if d == doc_id), 0)
        detailed_results_with_scores.append({"ID":result.id, "rrf_score": score, "payload": result.payload} )
    return detailed_results_with_scores


In [7]:
# id_ish_list = """

# 05453557-3177-4401-aba1-96f00a411dbf
# 096d4493-0efa-4c5f-9404-4c6ce2679643
# 1025840c-c2f9-48a3-a9d3-b0d4771e9d51
# 1071b8fe-18f1-47fe-94e5-b4abe06a789c
# 117edfae-1602-4c73-8c27-fb957db857bf
# 19788b7a-90df-453a-a57f-01855c6a6716
# 217687e9-f242-4ab0-9235-37655e674581
# 27bbb93e-60c7-49ec-934d-de0d3f9f5405
# 2942b4b6-7d0c-4188-8c1b-adc290ced19b
# 3182ad54-8fb1-407b-afeb-d5781fbde665
# 31ffa1fb-03f4-4061-b698-b5f2682fb294
# 3967c621-8cb0-450a-a10f-77e5ae860b4b
# 470af7e2-9bf1-4329-abed-92983e9ea367
# 55c8be63-7d2b-4fee-8a56-c64cf6bb5a8a
# 5705f999-8c9b-4780-be5c-7c430014da35
# 6064814c-b0e2-49df-b3c1-c43a13c65cf7
# 60a78362-da4b-4a67-b2c8-ee0d01d4ab07
# 610b35c1-657c-450b-8f60-5c46fb40317d
# 6268bc95-4154-4455-8017-be575e0815b8
# 6e0c1c4c-6f51-4992-888f-e2c71f11f7a7
# 70df9da7-e35f-4d44-ab22-5ae5bab45ad0
# 728b6302-dbc3-42ac-96e7-7fe725b2d139
# 72fc7443-0244-485a-8791-a5c3edcb4e80
# 73cc6698-c49f-4e65-84c5-13aec0a391ae
# 76bb4ed2-cdc3-4bcc-a655-3a5096d9b567
# 77c894d9-6731-479d-9bb4-a57a03d33791
# 7d061c07-5fb6-4544-8874-8443c5ccb870
# 866aed5d-76f9-49b8-a336-09a23a92ddee
# 92d64f8a-7745-41f0-9f13-cd1d489500ff
# 96fb9c0c-f52e-46c4-80c6-7d76a8e483e9
# 996a1489-6916-4e41-bde3-d0fcd9195c88
# 99ad6992-8d77-40cf-9b6c-e82950c2dff6
# ae3ca058-3ea6-400b-944d-32154bbd7942
# af0fcf27-34de-49d6-b380-a17a245c6205
# a14fb556-f79d-43ab-a844-c7d72526d61d
# 9dc1212e-5509-444c-a533-6f1b0083ce0f
# b9d23532-ee75-4f3c-bba0-f9c1373b28ee
# c6d2b01d-aeed-46fd-8126-8c2db6a87641
# c89a2c39-0c56-421e-900a-6c5bee4051c7
# cc67854b-0fc7-45f9-bd95-83dcbc10cc92
# ce85cf8d-620b-4b9f-8b2b-e67730e07a92
# cfc15e93-c8c3-459e-81cf-aa98d7092262
# d0014aaa-cae6-4a8b-9fd3-620ff0b4ae4c
# d1824846-c07c-4738-9494-38a32582f1b5
# d2bdb6bf-437b-4659-9c42-ab86300b0c85
# f42f1614-3010-440c-83df-70f5021029dd
# f1108da1-190a-4311-9195-b51b07af7f4f
# e3fb5b66-6dc2-4747-85ab-8795ef9258dd
# eafe45d2-70a0-4733-988a-96a72591330b
# bb99c695-02ea-44aa-8673-63341ccc5c96
# de2c7621-5139-4c1d-ba17-5b3d6c42422a
# de7df909-dfac-4b87-8d4c-d4028d8d903a
# da311aed-b54e-4c1d-b29a-d204e32eaa74
# eca2ef62-3277-4402-bd3c-4236be9c6e89
# ed77fd86-e25b-48c7-8925-525ab2410ea1
# f23e2cc5-ac0d-4f9a-a521-a9d6dc7761e3
# d343b5db-975b-4208-9a55-1c1cd4350967
# db474f2d-fc94-47f8-ac29-d1ca446f3f65
# ff22e622-1f35-431e-b032-d5b1c394ea0


# """

# # Convert the text into a list
# id_list = id_ish_list.strip().split("\n")

# # Output the list
# print(id_list)
# print (len(id_list))


['05453557-3177-4401-aba1-96f00a411dbf', '096d4493-0efa-4c5f-9404-4c6ce2679643', '1025840c-c2f9-48a3-a9d3-b0d4771e9d51', '1071b8fe-18f1-47fe-94e5-b4abe06a789c', '117edfae-1602-4c73-8c27-fb957db857bf', '19788b7a-90df-453a-a57f-01855c6a6716', '217687e9-f242-4ab0-9235-37655e674581', '27bbb93e-60c7-49ec-934d-de0d3f9f5405', '2942b4b6-7d0c-4188-8c1b-adc290ced19b', '3182ad54-8fb1-407b-afeb-d5781fbde665', '31ffa1fb-03f4-4061-b698-b5f2682fb294', '3967c621-8cb0-450a-a10f-77e5ae860b4b', '470af7e2-9bf1-4329-abed-92983e9ea367', '55c8be63-7d2b-4fee-8a56-c64cf6bb5a8a', '5705f999-8c9b-4780-be5c-7c430014da35', '6064814c-b0e2-49df-b3c1-c43a13c65cf7', '60a78362-da4b-4a67-b2c8-ee0d01d4ab07', '610b35c1-657c-450b-8f60-5c46fb40317d', '6268bc95-4154-4455-8017-be575e0815b8', '6e0c1c4c-6f51-4992-888f-e2c71f11f7a7', '70df9da7-e35f-4d44-ab22-5ae5bab45ad0', '728b6302-dbc3-42ac-96e7-7fe725b2d139', '72fc7443-0244-485a-8791-a5c3edcb4e80', '73cc6698-c49f-4e65-84c5-13aec0a391ae', '76bb4ed2-cdc3-4bcc-a655-3a5096d9b567',

In [ ]:
# Prompt template for rephrasing
updated_rephrase_question = PromptTemplate(template= """
You are tasked with rephrasing a user query for dense and sparse search on ISO and JRC documents. 
A good query should maximize precision and recall by focusing on relevant terms, avoiding conversational or reasoning context, and leveraging domain-specific language. 
Your goal is to create a concise, precise, and search-optimized query that retrieves relevant content effectively.



**User Query**:
{user_query}

**Instructions**:
1. Retain the meaning, context, and intent of the original query while simplifying it. Do not reduce it to a list of disconnected keywords.
2. Identify critical keywords or concepts in the user query (e.g., "storage," "ISO compliance," "capacity planning") and use them to construct a concise, structured query.
3. Write the rephrased query in the form of meaningful, concise phrases or sentences. Avoid generating disconnected terms or generic lists.
4. Expand the query with synonyms and related terms relevant to the domain (e.g., "storage systems," "capacity planning," "data protection") to improve hybrid search recall.
5. Tailor the query to support both dense (semantic) and sparse (keyword-based) retrieval by including specific phrases that enhance matching.
6. Avoid over-simplification or redundancy. Ensure the rephrased query is meaningful and aligned with the original user query.

**Example Rephrasing**:
User Query: "What are the best practices and ISO guidelines for datacenter storage management in a new facility?"
Bad Output: "Datacenter storage, ISO compliance, best practices."
Good Output: "Best practices and ISO guidelines for datacenter storage management in new facilities."
If ISO or JRC are not mentioned do not mention it either

**Search Query**:                                 
Please provide a concise and precise search query based on the above instructions.                                   
                                   
         
                                   """)

# Function to rephrase the user query with template
def updated_rephase_query(user_query):
   new_quest = updated_rephrase_question.format(user_query=user_query)
   new_query = Chat_llm.invoke(new_quest)
    
   return new_query

In [ ]:
# Remake the query for multiple iterations
def updated_query_remake(prompt, iterations):
    all_repharased_data = []
    
    for i in range(iterations):
        new_query = updated_rephase_query(prompt)
        all_repharased_data.append(new_query.content)
        
        
        
    return all_repharased_data



In [ ]:
# Get retrieved data for multiple iterations at n_chunks for original query
def query_remake_experiment(prompt, iterations, n_chunks):
    all_repharased_data = []
    
    for i in range(iterations):
        new_query = updated_rephase_query(prompt)
        dense = merged_dense_for_RRF(new_query.content, n_chunks)
        sparse = merged_sparse_for_RRF(new_query.content, n_chunks)
        
        fused_results = reciprocal_rank_fusion(dense, sparse, k=60)
        nodes_rrf_back = Fetch_chunks_back(fused_results)
        
        data = (i+1, new_query.content, dense, sparse, nodes_rrf_back[:n_chunks])
        all_repharased_data.append(data)
        
        
        
    return all_repharased_data


In [ ]:
# Template for LLM final generation 
costom_template = PromptTemplate(
    template="""
You are a highly knowledgeable assistant specializing in the JRC assessment framework (JRC code of conduct) and ISO standards (50001, 14001, 30134). 
Your task is to generate a developer-focused framework by interpreting relevant chunks from the provided content. Follow these strict instructions:




1. **Focus on Developer Needs**:
    - Provide actionable technical requirements for developers.
    - Ensure the requirements are framed in plain, developer-friendly language.

2. **Structure**:
    - Include clear headings for each requirement category.
    - Break down requirements into:
        - Title
        - Technical Details (if stated in chunk) - 
                The technical details should be compliant with ISO standard directives part 2 chapter 7. Make sure the technical details are stated as requirements. Follow these rules:
                1. For mandatory provisions, use "shall." Example: "The system shall provide a log of all user activities."
                2. For advisory provisions, use "should." Example: "The interface should include a dark mode option for improved user comfort."
                3. For permissible actions, use "may." Example: "Users may configure their own notification preferences."
                4. For statements about potential or capability, use "can." Example: "The system can handle up to 10,000 simultaneous connections."
        - Evidence Required (if stated in chunk)
        - Reference Frameworks (if applicable) and Source (ISO/JRC section)
        - Type of requirement (mandatory, non-mandatory (advisory), permissible etc. for iso and Whole datacenter, new build, existing building, etc. and importance value for JRC)
        

3. **Comprehensiveness**:
    - Use **all relevant chunks** to create the framework, even if their relevance is marginal.
    -Mention which chunks you did not use and why.
    - Summarize overlapping or repetitive chunks to avoid redundancy but retain all key details.

4. **Avoid Assumptions**:
    - Only use information explicitly found in the chunks.
    - Do not introduce new concepts or requirements not in the source content.

5. **Formatting**:
    - Present the framework in a structured markdown format.
    - Use numbered sections and subsections for easy navigation.
    

6. **Metadata Inclusion**:
    - For each requirement, include references such as ISO/JRC standards, specific practice numbers, and evidence.

**Context**:
These chunks are sorted by relevance using the RRF method and pertain to the query.

**Chunks**:
{chunks}

**Question**:
{question}

**Answer**:
Provide a comprehensive developer-oriented framework meeting the above criteria.
"""
)


        
# Fetch the chunks for the conversational system
def fetch_chunks_rrf_sorted(user_question, limitN):
    dense_results = merged_dense_for_RRF(user_question, limitN)
    # print(dense_results)
    sparse_results = merged_sparse_for_RRF(user_question, limitN)
    # print(sparse_results)
    fused_results = reciprocal_rank_fusion(dense_results, sparse_results, k=60)
    Nodes_rrf_back = Fetch_chunks_back(fused_results)
    return Nodes_rrf_back


# Retrieve the chunks, put them in the template and generate the final response
def conversational_system(query):
   
    # Fetch context from the fused results
    new_query = updated_rephase_query(query)
    context = fetch_chunks_rrf_sorted(new_query.content, 25)
    print("Query used: ", new_query)
    

    
    chunks1 = context[:15]

    print(chunks1)
    
    custom_prompt = costom_template.format(chunks=chunks1, question=query)
    response = Chat_llm.invoke(custom_prompt)
    print(response)
 
    
    print(len(chunks1))
    
    print("Your query: ", query)
    return response , chunks1






In [ ]:
# List used to generate responses

queries_non_list = """
How can I reduce cooling costs without breaking compliance?



"""

# Convert the text into a list
queries_list = queries_non_list.strip().split("\n")

# Output the list
print(queries_list)
print (len(queries_list))



['How can I reduce cooling costs without breaking compliance?']
1


In [ ]:
# def get_responses_for_query(query, k):
#     responses = []
#     save_path = "responses.json"
    
#     for i in range(k):
#         start_time = time.time()
#         response, context = conversational_system(query)
#         end_time = time.time()
#         time_taken = end_time - start_time
#         print("All_info")
#         print(query, response, context, time_taken)
#         responses.append((response, context, time_taken))
        

        
#     time.sleep(5)
    
#     return responses


In [ ]:
# Time and save the generated responses in a CSV file iteratively

def get_responses_for_query(query, k):
    responses = []
    save_path = "responses.csv"  
    file_exists = os.path.isfile(save_path)

    # Open CSV file in append mode
    with open(save_path, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write header only if file does not exist
        if not file_exists:
            writer.writerow(["Query", "Response", "Context", "Time Taken"])  
        
        for i in range(k):
            start_time = time.time()
            response, context = conversational_system(query)
            end_time = time.time()
            time_taken = end_time - start_time
            
            print("All_info")
            print(query, response, context, time_taken)
            
            # Save the response to CSV immediately
            writer.writerow([query, response, context, time_taken])
            
            responses.append((response, context, time_taken))
            
            time.sleep(5)  # Pause between iterations (Helps with API eventhough it should not be needed)

    return responses


In [ ]:
# Iterate through the queries and get the responses
def list_of_query_responses(queries, k):
    responses_all = {}
    
    
    for query in queries:
        response_new = get_responses_for_query(query, k)
        if query not in responses_all:
            responses_all[query] = []  # Ensure the key exists
        responses_all[query].extend(response_new)
        

    return responses_all

In [ ]:
# Run the experiment
try:
    all_query_responses = list_of_query_responses(queries_list, 1)
except Exception as e:
    print(f"Error occurred: {e}")

Query used:  content='Reducing cooling costs while maintaining regulatory compliance.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 384, 'total_tokens': 394, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None} id='run-9d6a1545-d399-478b-bae0-f7483b9316bc-0' usage_metadata={'input_tokens': 384, 'output_tokens': 10, 'total_tokens': 394, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
[{'ID': '31ffa1fb-03f4-4061-b698-b5f2682fb294', 'rrf_score': 0.03200204813108039, 'payload': {'content': 'CoC No: 5.4.2.5, Name: Select systems which facilitate the use of \x93Free Cooling\x94, Description: Cooling d

In [ ]:
#put all data in DF (Though its also in the csv)

data = []
for query, responses in all_query_responses.items():
    for response, context, time_taken in responses:
        data.append((query, response, context, time_taken))
df = pd.DataFrame(data, columns=["Query", "Response", "Context", "Time Taken (s)"])


In [ ]:
df

,Query,Response,Context,Time Taken (s)
0,How do I upgrade an existing data center for b...,content='# Developer-Oriented Framework for Im...,[{'ID': 'fd9d5008-29bd-4f5b-8566-eb9d4fbec641'...,21.370046
1,How do I upgrade an existing data center for b...,content='# Developer-Focused Framework for Upg...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,13.496635
2,How do I upgrade an existing data center for b...,content='# Developer-Focused Framework for Upg...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,19.420900
3,How do I upgrade an existing data center for b...,"content=""# Developer-Focused Framework for Dat...",[{'ID': '658a771e-0f82-4184-94f9-448b84777922'...,31.448245
4,How do I upgrade an existing data center for b...,content='### Developer-Oriented Framework for ...,[{'ID': 'fd9d5008-29bd-4f5b-8566-eb9d4fbec641'...,22.402977
5,What are the risks of retrofitting an older da...,"content=""```markdown\n# Developer-Oriented Fra...",[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,19.001353
6,What are the risks of retrofitting an older da...,content='# Developer-Oriented Framework for Re...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,26.780527
7,What are the risks of retrofitting an older da...,"content=""# Developer-Oriented Framework for Da...",[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,25.360740
8,What are the risks of retrofitting an older da...,"content=""Here's a structured, developer-focuse...",[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,14.257893
9,What are the risks of retrofitting an older da...,content='# Developer-Focused Framework for Dat...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,22.206357


In [ ]:
# save the DF to a CSV file
df.to_csv("conversational_system_responses2.csv")

In [ ]:
#  For faithfulness calculation

df = pd.read_csv("responses_queries.csv")

# Convert JSON-like strings in "Response" column to dictionary objects
def convert_to_object(value):
    try:
        return json.loads(value) if isinstance(value, str) and value.startswith("{") else value
    except json.JSONDecodeError:
        return value  # Keep original if parsing fails

df["Response"] = df["Response"].apply(convert_to_object)

# Verify column type
print(df["Response"].dtype)  # Should show 'object' (dicts inside)


object


In [15]:
# file_path = "responses3_all content cleaned.csv"
# df = pd.read_csv(file_path, encoding="latin1")

In [ ]:
# Extract the details out of context row

def extract_contents(context):
    contents = []
    try:
        # Convert string to dictionary/list if needed
        context_dict = ast.literal_eval(context) if isinstance(context, str) else context

        # If context is a list of dictionaries
        if isinstance(context_dict, list):
            for item in context_dict:
                if isinstance(item, dict) and 'payload' in item and 'content' in item['payload']:
                    contents.append(item['payload']['content'])
        
        # If context is a single dictionary
        elif isinstance(context_dict, dict):
            if 'payload' in context_dict and 'content' in context_dict['payload']:
                contents.append(context_dict['payload']['content'])

    except Exception as e:
        print(f"Error parsing context: {e}")
    
    return contents  # Return a list of extracted contents

# Print extracted contents
df["Extracted Contents"] = df["Context"].apply(extract_contents)
df


,Unnamed: 0,Query,Response,Context,Time Taken (s),Extracted Payloads,Extracted Contents
0,0.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Da...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,30.539217,[A.2 Data\tcentre\tavailability\tobjectives\nT...,[A.2 Data\tcentre\tavailability\tobjectives\nT...
1,1.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Op...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,38.904418,[A.2 Data\tcentre\tavailability\tobjectives\nT...,[A.2 Data\tcentre\tavailability\tobjectives\nT...
2,2.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Da...,[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,87.436647,"[CoC No: 4.1.6, Name: IT equipment power usage...","[CoC No: 4.1.6, Name: IT equipment power usage..."
3,3.0,How do I optimize my data center’s power distr...,"content=""# Developer-Oriented Framework for Da...",[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,31.669091,"[CoC No: 4.1.6, Name: IT equipment power usage...","[CoC No: 4.1.6, Name: IT equipment power usage..."
4,4.0,How do I optimize my data center’s power distr...,content='# Developer-Focused Framework for Dat...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,55.313114,[A.2 Data\tcentre\tavailability\tobjectives\nT...,[A.2 Data\tcentre\tavailability\tobjectives\nT...
5,5.0,How do I optimize my data center’s power distr...,content='```markdown\n# Developer-Focused Ener...,[{'ID': '8f206e3a-2233-4dcd-b8fd-3320b3b00535'...,27.011177,[A.6.6\nPlanning for collection of energy data...,[A.6.6\nPlanning for collection of energy data...
6,6.0,How can I monitor and report energy consumptio...,"content=""Here's a structured developer-oriente...",[{'ID': 'fd9d5008-29bd-4f5b-8566-eb9d4fbec641'...,24.452488,"[CoC No: 3.2.8, Name: Sustainable energy usage...","[CoC No: 3.2.8, Name: Sustainable energy usage..."
7,7.0,How can I monitor and report energy consumptio...,"content=""```markdown\n# Developer-Oriented Fra...",[{'ID': '12636a3d-2a8f-4bfd-b6c9-61bc49367856'...,32.049315,"[CoC No: 9.3.1, Name: Written Reporting, Descr...","[CoC No: 9.3.1, Name: Written Reporting, Descr..."
8,8.0,How can I monitor and report energy consumptio...,content='# Developer-Focused Framework for Mon...,[{'ID': '12636a3d-2a8f-4bfd-b6c9-61bc49367856'...,17.838404,"[CoC No: 9.3.1, Name: Written Reporting, Descr...","[CoC No: 9.3.1, Name: Written Reporting, Descr..."
9,9.0,How can I monitor and report energy consumptio...,content='```markdown\n# Developer-Oriented Fra...,[{'ID': '7eb7238e-ec78-49ef-a3a1-d052faf63db4'...,15.325679,"[CoC No: 7.1.4, Name: Energy & temperature rep...","[CoC No: 7.1.4, Name: Energy & temperature rep..."


In [ ]:
# Extract the payloards out of context row
def extract_payload_strings(context):
    payload_strings = []
    try:
        # Convert string to dictionary/list if needed
        context_dict = ast.literal_eval(context) if isinstance(context, str) else context

        # Helper function to recursively collect string values
        def collect_strings(data):
            if isinstance(data, str):
                payload_strings.append(data)
            elif isinstance(data, dict):
                for value in data.values():
                    collect_strings(value)
            elif isinstance(data, list):
                for item in data:
                    collect_strings(item)

        # If context is a list of dictionaries
        if isinstance(context_dict, list):
            for item in context_dict:
                if isinstance(item, dict) and 'payload' in item:
                    collect_strings(item['payload'])

        # If context is a single dictionary
        elif isinstance(context_dict, dict) and 'payload' in context_dict:
            collect_strings(context_dict['payload'])

    except Exception as e:
        print(f"Error parsing context: {e}")

    return payload_strings  # Return a list of extracted string values

# Apply function to the dataframe
df["Extracted Payloads"] = df["Context"].apply(extract_payload_strings)


In [19]:
df

,Unnamed: 0,Query,Response,Context,Time Taken (s),Extracted Payloads
0,0.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Da...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,30.539217,[A.2 Data\tcentre\tavailability\tobjectives\nT...
1,1.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Op...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,38.904418,[A.2 Data\tcentre\tavailability\tobjectives\nT...
2,2.0,How do I optimize my data center’s power distr...,content='# Developer-Oriented Framework for Da...,[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,87.436647,"[CoC No: 4.1.6, Name: IT equipment power usage..."
3,3.0,How do I optimize my data center’s power distr...,"content=""# Developer-Oriented Framework for Da...",[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,31.669091,"[CoC No: 4.1.6, Name: IT equipment power usage..."
4,4.0,How do I optimize my data center’s power distr...,content='# Developer-Focused Framework for Dat...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,55.313114,[A.2 Data\tcentre\tavailability\tobjectives\nT...
5,5.0,How do I optimize my data center’s power distr...,content='```markdown\n# Developer-Focused Ener...,[{'ID': '8f206e3a-2233-4dcd-b8fd-3320b3b00535'...,27.011177,[A.6.6\nPlanning for collection of energy data...
6,6.0,How can I monitor and report energy consumptio...,"content=""Here's a structured developer-oriente...",[{'ID': 'fd9d5008-29bd-4f5b-8566-eb9d4fbec641'...,24.452488,"[CoC No: 3.2.8, Name: Sustainable energy usage..."
7,7.0,How can I monitor and report energy consumptio...,"content=""```markdown\n# Developer-Oriented Fra...",[{'ID': '12636a3d-2a8f-4bfd-b6c9-61bc49367856'...,32.049315,"[CoC No: 9.3.1, Name: Written Reporting, Descr..."
8,8.0,How can I monitor and report energy consumptio...,content='# Developer-Focused Framework for Mon...,[{'ID': '12636a3d-2a8f-4bfd-b6c9-61bc49367856'...,17.838404,"[CoC No: 9.3.1, Name: Written Reporting, Descr..."
9,9.0,How can I monitor and report energy consumptio...,content='```markdown\n# Developer-Oriented Fra...,[{'ID': '7eb7238e-ec78-49ef-a3a1-d052faf63db4'...,15.325679,"[CoC No: 7.1.4, Name: Energy & temperature rep..."


In [ ]:
# # Calculate faithfulness scores for each response for all 4 scenarios
# save_path = "responses.csv"  # Save responses in a CSV file
# file_exists = os.path.isfile(save_path)

# results = []

# for index,row in df.iterrows():   
#     query = (row["Query"])
#     response = (row["Response"])
#     response_only_techncial = (row["Reformated to only technical details"]) # This was done manually 
#     context = (row["Extracted Contents"]) 
#     print("User: ", query, response, response_only_techncial, context) 
    
#     sample1 = SingleTurnSample(
#         user_input=query,
#         response=response,
#         retrieved_contexts=context
#         )
    
#     sample2 = SingleTurnSample(
#         user_input=query,
#         response=response_only_techncial,
#         retrieved_contexts=context
#         )
    
#     scorer = Faithfulness(llm=evaluator_llm)
#     faithfulness_original =  await scorer.single_turn_ascore(sample1)
#     faithfulness_only_technical =  await scorer.single_turn_ascore(sample2)
#     print("Faithfulness Original: ", faithfulness_original)
#     print("Faithfulness Only Technical: ", faithfulness_only_technical
    
#     results.append({
#             "Query": query,
#             "Response": response,
#             "Response_Only_Technical": response_only_techncial,
#             "Context": context,
#             "Faithfulness_Original": faithfulness_original,
#             "Faithfulness_Only_Technical": faithfulness_only_technical
#         })
#     time.sleep(3)
    
    

    

SyntaxError: '(' was never closed (1873739272.py, line 29)

In [20]:
df["Extracted Payloads"][0]

['A.2 Data\tcentre\tavailability\tobjectives\nThe availability objectives of a data centre can restrict such improvements due to the use of redundant infrastructure components and infrastructures necessary to respond the following circumstances: a) the user of the data centre will either set or demand a series of service level agreements (SLAs) based upon the criticality of their business case covering continuity and quality of power and environmental control provision, connectivity and security; b) the SLAs will reflect directly the appetite for risk that the user has and their ability or willingness to accept shutdowns for maintenance and repairs in the critical systems; c) data centres whose users can accept high levels of risk, scheduled shutdowns for maintenance and shutdowns for fault repair can reach very low design KPIs; d) data centres whose users cannot accept risk of failure and demand concurrent maintenance and inherent fault tolerance are limited by those SLAs to a higher 

In [22]:
df["Extracted Contents"][0]

['A.2 Data\tcentre\tavailability\tobjectives\nThe availability objectives of a data centre can restrict such improvements due to the use of redundant infrastructure components and infrastructures necessary to respond the following circumstances: a) the user of the data centre will either set or demand a series of service level agreements (SLAs) based upon the criticality of their business case covering continuity and quality of power and environmental control provision, connectivity and security; b) the SLAs will reflect directly the appetite for risk that the user has and their ability or willingness to accept shutdowns for maintenance and repairs in the critical systems; c) data centres whose users can accept high levels of risk, scheduled shutdowns for maintenance and shutdowns for fault repair can reach very low design KPIs; d) data centres whose users cannot accept risk of failure and demand concurrent maintenance and inherent fault tolerance are limited by those SLAs to a higher 

In [ ]:
# Calculate faithfulness scores for each response for all 4 scenarios

save_path = "faith_results_with_metadata.csv"  # Save responses in a CSV file
file_exists = os.path.isfile(save_path)

results = []

# Open CSV file in append mode
with open(save_path, mode="a", newline="", encoding="utf-8") as file:
    fieldnames = ["Query", "Response", "Response_Only_Technical", "Context_and_metadata", "Faithfulness_Original", "Faithfulness_Only_Technical"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write header only if file does not exist
    if not file_exists:
        writer.writeheader()

    for index, row in df.iterrows():
        query = row["Query"]
        response = row["Response"]
        response_only_technical = row["Reformated to only technical details"] # This was done manually 
        context = row["Extracted Payloads"]

        print("User:", query, response, response_only_technical, context)

        sample1 = SingleTurnSample(
            user_input=query,
            response=response,
            retrieved_contexts=context
        )

        sample2 = SingleTurnSample(
            user_input=query,
            response=response_only_technical,
            retrieved_contexts=context
        )

        scorer = Faithfulness(llm=evaluator_llm)
        faithfulness_original = await scorer.single_turn_ascore(sample1)
        faithfulness_only_technical = await scorer.single_turn_ascore(sample2)

        print("Faithfulness Original:", faithfulness_original)
        print("Faithfulness Only Technical:", faithfulness_only_technical)

        # Write to CSV immediately
        writer.writerow({
            "Query": query,
            "Response": response,
            "Response_Only_Technical": response_only_technical,
            "Context_and_metadata": context,
            "Faithfulness_Original": faithfulness_original,
            "Faithfulness_Only_Technical": faithfulness_only_technical
        })
        
        results.append({
            "Query": query,
            "Response": response,
            "Response_Only_Technical": response_only_technical,
            "Context_and_metadata": context,
            "Faithfulness_Original": faithfulness_original,
            "Faithfulness_Only_Technical": faithfulness_only_technical
        })

        file.flush()  # Ensure data is written to the file immediately

        time.sleep(3)  # Pause between iterations if needed


User: How can I reduce cooling costs without breaking compliance? content='# Developer-Oriented Framework for Cooling Cost Reduction in Data Centers\n\n## 1. Regular Maintenance Practices\n\n### 1.1 Implement Effective Regular Maintenance of Cooling Plant\n- **Technical Details**: The system shall ensure regular maintenance to achieve or conserve a "like new condition" which includes maintenance operations such as belt tension, condenser coil fouling, and evaporator fouling. Regular filter changes and cleaning regimes are essential.\n- **Evidence Required**: Maintenance and service reports.\n- **Reference Frameworks**: JRC Assessment Framework, CoC No: 5.2.7.\n- **Type of Requirement**: Mandatory (Whole Data Center, Priority/Value: 2).\n\n## 2. Cooling System Design Considerations\n\n### 2.1 Indirect Water Free Cooling with CRAH and Dry Cooler\n- **Technical Details**: The infrastructure shall consider a design that allows chilled water to be cooled by external ambient air using free c

In [ ]:
# 

In [135]:
# df

,Query,Response,Context,Time Taken,Unnamed: 4,Reformated to only technical details,faithfulness original response,faitfulness only technical,Payload_Content
0,How can I reduce cooling costs without breakin...,content='# Developer-Oriented Framework for Co...,[{'ID': '096d4493-0efa-4c5f-9404-4c6ce2679643'...,21.689507,NaN,1. Regular Maintenance Practices\n1.1 Implemen...,NaN,NaN,None
1,How can I reduce cooling costs without breakin...,content='# Developer-Oriented Framework for Re...,[{'ID': '31ffa1fb-03f4-4061-b698-b5f2682fb294'...,21.190926,NaN,1. Free Cooling Systems\n1.1 Select Systems Fa...,0.55102,1.0,None
2,How can I reduce cooling costs without breakin...,content='```markdown\n# Developer-Focused Fram...,[{'ID': '096d4493-0efa-4c5f-9404-4c6ce2679643'...,15.534033,NaN,## 1. Effective Regular Maintenance of Cooling...,NaN,NaN,None
3,How can I reduce cooling costs without breakin...,content='# Developer-Oriented Framework for Re...,[{'ID': 'bb99c695-02ea-44aa-8673-63341ccc5c96'...,20.685196,NaN,1. Cooling System Management\n1.1 Achieved Fre...,NaN,NaN,None
4,Whatâs the best way to forecast my data cent...,"content=""Here is a developer-focused framework...",[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,34.742501,NaN,## 1. IT Equipment and Software Requirements #...,NaN,NaN,None
5,Whatâs the best way to forecast my data cent...,content='Below is a developer-focused framewor...,[{'ID': '44a91a1e-ddaa-4e79-a008-841be4eebc30'...,29.711448,NaN,## 1. Efficient Software Selection and Develop...,NaN,NaN,None
6,Whatâs the best way to forecast my data cent...,"content=""# Developer-Oriented Framework for Da...",[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,25.034527,NaN,1. IT Equipment Selection\n1.1 IT Equipment Po...,NaN,NaN,None
7,Whatâs the best way to forecast my data cent...,content='```markdown\n# Developer-Oriented Fra...,[{'ID': '84970a49-031c-466e-a29b-3145e79d1ef3'...,19.520759,NaN,.1 System Requirements\nThe design shall consi...,NaN,NaN,None
8,Whatâs the best way to forecast my data cent...,"content=""```markdown\n# Developer-Focused Fram...",[{'ID': '44a91a1e-ddaa-4e79-a008-841be4eebc30'...,36.140016,NaN,# Technical Details Extracted ## 1. Software E...,NaN,NaN,None
9,How can I improve my data centerâs uptime wi...,"content=""Below is a developer-focused framewor...",[{'ID': '19788b7a-90df-453a-a57f-01855c6a6716'...,17.530441,NaN,1. IT Equipment and Environmental Management\n...,NaN,NaN,None
